In [4]:
import pandas as pd

import sqlalchemy
import config
import glob

In [5]:
engine = sqlalchemy.create_engine("sqlite:///../data/luftdaten_dwd.db")

"station_code" TEXT, 
"pollutant" TEXT,
"datehour" TEXT, 
"value" BIGINT,


df_db = pd.read_sql(('select "station_code","datehour","pollutant" from "dwd_data_1H" '
                     'where "station_code" LIKE %(s_code)s AND "datehour"  LIKE %(dthour)s AND "pollutant" LIKE %(pollutant)s ),
                   engine,params={"station_code":''})

In [22]:
all_files = glob.glob('../data/dwd_to_be_inserted/*.csv')

In [23]:
all_files

['../data/dwd_to_be_inserted/dwd_data_1H_2.csv',
 '../data/dwd_to_be_inserted/dwd_data_1H_0.csv',
 '../data/dwd_to_be_inserted/dwd_data_1H_6.csv',
 '../data/dwd_to_be_inserted/dwd_data_1H_5.csv',
 '../data/dwd_to_be_inserted/dwd_data_1H_4.csv',
 '../data/dwd_to_be_inserted/dwd_data_1H_1.csv',
 '../data/dwd_to_be_inserted/dwd_data_1H_3.csv']

In [31]:
for f in all_files :
    df = pd.read_csv(f)
    df.pollutant='PM10'
    df.to_sql('dwd_data_1H',engine,index=False, if_exists='append')
    print(f'saved {f}' )


saved ../data/dwd_to_be_inserted/dwd_data_1H_2.csv
saved ../data/dwd_to_be_inserted/dwd_data_1H_0.csv
saved ../data/dwd_to_be_inserted/dwd_data_1H_6.csv
saved ../data/dwd_to_be_inserted/dwd_data_1H_5.csv
saved ../data/dwd_to_be_inserted/dwd_data_1H_4.csv
saved ../data/dwd_to_be_inserted/dwd_data_1H_1.csv
saved ../data/dwd_to_be_inserted/dwd_data_1H_3.csv


In [32]:
engine.dispose()

In [13]:
def clean_and_store_dwd_data(pollutant='PM10'):
    chunksize = 10**6
    i=0
    for chunk in pd.read_sql('dwd_data_PM10_1SMW',engine,chunksize=chunksize):
        print(f'new chunk loaded : {chunk.shape}')
        df = chunk
        df= df[['Stationscode', 'Zeit', 'Messwert (in µg/m³)']]
        df.columns = ['station_code','datehour','value']
        print(df.columns)
        df['pollutant']= pollutant
        df = df[['station_code', 'pollutant', 'datehour', 'value']]    
        df.drop_duplicates(inplace=True) 
        
        print('starting saving df')
        df.to_csv(f'dwd_data_1H_{str(i)}.csv',index=False)
        i=i+1
        #df.to_sql('dwd_data_1H',engine,index=False, if_exists='append')
        print('done with saving df')

In [14]:
clean_and_store_dwd_data(engine)

new chunk loaded : (1000000, 8)
Index(['station_code', 'datehour', 'value'], dtype='object')


/home/ramon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


starting saving df
done with saving df
new chunk loaded : (1000000, 8)
Index(['station_code', 'datehour', 'value'], dtype='object')
starting saving df
done with saving df
new chunk loaded : (1000000, 8)
Index(['station_code', 'datehour', 'value'], dtype='object')
starting saving df
done with saving df
new chunk loaded : (1000000, 8)
Index(['station_code', 'datehour', 'value'], dtype='object')
starting saving df
done with saving df
new chunk loaded : (1000000, 8)
Index(['station_code', 'datehour', 'value'], dtype='object')
starting saving df
done with saving df
new chunk loaded : (1000000, 8)
Index(['station_code', 'datehour', 'value'], dtype='object')
starting saving df
done with saving df
new chunk loaded : (915806, 8)
Index(['station_code', 'datehour', 'value'], dtype='object')
starting saving df
done with saving df


In [10]:
df.index[-1]

999999

In [ ]:
#df.to_sql('dwd_data_1H',engine,index=False, if_exists='append')

In [ ]:
#engine.dispose()

In [ ]:
df_current_db = pd.read_sql('dwd_data_1H',engine)

In [ ]:
df_current_db